In [1]:
import pandas as pd
import numpy as np
import os
from difflib import SequenceMatcher
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

# Data  

In [2]:
results_dir = 'C:\\Users\\RonyArmon\\Projects_Code\\Cluster_Activities\\results'
embed_dir = 
pos_df = pd.read_pickle(os.path.join(results_dir,'pos.pkl'))
tokens_similarity = pd.read_pickle(os.path.join(results_dir,'tokens_similarity.pkl'))
lemmas_synonyms = pd.read_excel(os.path.join(results_dir, 'lemmas_synonyms.xlsx'))
embedding_model_names = ['glove-twitter-25', 'word2vec-google-news-300']
similarity_matrices = {}
for name in embedding_model_names:
    similarity_matrices[name] = pd.read_pickle(os.path.join(results_dir, 'similarity/embeddings/twitter_glove.pkl')) 

# Tokens with Ids

In [12]:
id1, id2 = 3,4
df1 = pos_df[pos_df['name_id']==id1]
df2 = pos_df[pos_df['name_id']==id2]
tokens_ids1 = list(zip(df1['token'], df1['token_id']))
tokens_ids2 = list(zip(df2['token'], df2['token_id']))
if len(tokens_ids1)>len(tokens_ids2):
    tokens_ids1_holder = tokens_ids1
    tokens_ids1 = tokens_ids2
    tokens_ids2 = tokens_ids1_holder
print('short name tokens:', len(tokens_ids1), tokens_ids1)
print('long name tokens:', len(tokens_ids2), tokens_ids2)

short name tokens: 7 [('site', 16), ('drainage', 17), ('a&a', 18), ('works', 19), ('&', 20), ('site', 21), ('demob', 22)]
long name tokens: 7 [('turbine', 23), ('hall', 24), ('(', 25), ('pre-bore', 26), ('socket', 27), ('h-piles', 28), (')', 29)]


# Pairs Scores   

In [41]:
lemmas_synonyms.head()

,lemma,synonym
0,once,erstwhile
1,once,formerly
2,once,erst
3,address,speech
4,address,speak


In [43]:
token_pairs = []
for token_id1 in tokens_ids1:
    score = 0
    token1, id1 = token_id1
    checked, scores = [], {}
    for token_id2 in tokens_ids2:
        token2, id2 = token_id2
        if id2 not in checked:
            
            # String similarity score 
            score += tokens_similarity.at[token1, token2]
            
            # Synonym score 
            lemma1 = pos_df['lemma'][pos_df['token_id']==id1].values[0]
            lemma2 = pos_df['lemma'][pos_df['token_id']==id2].values[0]
            lemma1_syns = list(lemmas_synonyms['synonym'][lemmas_synonyms['lemma']==lemma1])
            lemma2_syns = list(lemmas_synonyms['synonym'][lemmas_synonyms['lemma']==lemma2])
            lemmas_12_syns = set(lemma1_syns).intersection(set(lemma2_syns))
            if len(lemmas_12_syns)>1: score += 1
                        
            
            # Read and score by embeddings similarity 
            
            checked.append(id2)
            #print(token1, id1, token2,id2, score)
            # Filter score
            if score>0.3: 
                scores[(token2, id2)]=score
    scores_array = np.array(list(scores.values()))
    if np.sum(scores_array)>0:
        # Filter score
        max_score = np.max(scores_array) 
        match_score = {k:v for k,v in scores.items() if v == max_score}
        ms_keys, ms_vals = list(match_score.keys()), list(match_score.values())
        match_score = [id1, ms_keys[0][1], ms_vals[0]]
    else:
        match_score = ()
    token_pairs.append(match_score)
    #print('match score:', match_score)
token_pairs = [t for t in token_pairs if t]
print('token_pairs:', token_pairs)

lemma1_syns: ['situation', 'place', 'locate', 'website']
lemma2_syns: []
lemmas_12_syns: set()
lemma1_syns: ['situation', 'place', 'locate', 'website']
lemma2_syns: ['hallway', 'anteroom', 'mansion', 'manse', 'dormitory', 'lobby', 'antechamber', 'dorm', 'foyer', 'residence', 'vestibule']
lemmas_12_syns: set()
lemma1_syns: ['situation', 'place', 'locate', 'website']
lemma2_syns: []
lemmas_12_syns: set()
lemma1_syns: ['situation', 'place', 'locate', 'website']
lemma2_syns: []
lemmas_12_syns: set()
lemma1_syns: ['situation', 'place', 'locate', 'website']
lemma2_syns: []
lemmas_12_syns: set()
lemma1_syns: ['situation', 'place', 'locate', 'website']
lemma2_syns: []
lemmas_12_syns: set()
lemma1_syns: ['situation', 'place', 'locate', 'website']
lemma2_syns: []
lemmas_12_syns: set()
lemma1_syns: ['drain']
lemma2_syns: []
lemmas_12_syns: set()
lemma1_syns: ['drain']
lemma2_syns: ['hallway', 'anteroom', 'mansion', 'manse', 'dormitory', 'lobby', 'antechamber', 'dorm', 'foyer', 'residence', 'vesti

# Part of Speech Scores (LOCpi, POSpi)  

In [38]:
pos_scores = []
for pair in token_pairs:
    pos_score = 0
    print('------')
    print(pair)
    id1, id2 = pair[0], pair[1]
    pos1 = pos_df['upos'][pos_df['token_id']==id1].values[0]
    pos2 = pos_df['upos'][pos_df['token_id']==id2].values[0]
    print('upos:', pos1, pos2)
    if pos1 == pos2: pos_score = 1
    pair.append(pos_score)
    print(pair)
    pos_scores.append(pair)
    pair = []
print(pos_scores)

------
[16, 27, 0.4, 1]
upos: NOUN NOUN
[16, 27, 0.4, 1, 1]
------
[17, 23, 0.53, 1]
upos: NOUN NOUN
[17, 23, 0.53, 1, 1]
------
[19, 27, 0.36, 1]
upos: NOUN NOUN
[19, 27, 0.36, 1, 1]
------
[21, 27, 0.4, 1]
upos: NOUN NOUN
[21, 27, 0.4, 1, 1]
------
[22, 26, 0.31, 0]
upos: PUNCT ADJ
[22, 26, 0.31, 0, 0]
[[16, 27, 0.4, 1, 1], [17, 23, 0.53, 1, 1], [19, 27, 0.36, 1, 1], [21, 27, 0.4, 1, 1], [22, 26, 0.31, 0, 0]]


In [24]:
print(pos_df.head())

   token_id  name_id loc      token      lemma   upos xpos head deprel  \
0         1        1   1  construct  construct   VERB   VB    0   root   
0         2        1   2     trials      trial   NOUN  NNS    1    obj   
0         3        1   3          &          &  CCONJ   CC    4     cc   
0         4        1   4       test       test   NOUN   NN    2   conj   
0         5        1   5        for        for    ADP   IN    7   case   

  start_char end_char                  feats  
0          0        9  Mood=Imp|VerbForm=Fin  
0         10       16            Number=Plur  
0         17       18                      _  
0         19       23            Number=Sing  
0         24       27                      _  


In [ ]:
pos_scores = []
for pair in token_pairs:
    pos_score = 0
    print('------')
    print(pair)
    id1, id2 = pair[0], pair[1]
    pos1 = pos_df['upos'][pos_df['token_id']==id1].values[0]
    pos2 = pos_df['upos'][pos_df['token_id']==id2].values[0]
    print('upos:', pos1, pos2)
    if pos1 == pos2: pos_score = 1
    pair.append(pos_score)
    print(pair)
    pos_scores.append(pair)
    pair = []
print(pos_scores)